# MP 7: Learning grasps

**Due date**: March 20, 2021 at 10:45am.

**Instructions**: Read and complete the problems below. In this assignment, you should be switched over to a local install. 

To submit your assignment, perform the following:

1. Double-check that your programs run without error.
2. Submit this file, all of your .py files, and some illustrative predicted images (just the ones referenced in your written answers) on Moodle [http:/learn.illinois.edu](http:/learn.illinois.edu).
3. If you are using any external libraries other than the ones that are indicated during the installation process, include a README file indicating which library and version you are using.  To be on the safe side, you should include a backup procedure until the graders verify that they are able to support use of that library.

## Problem 1: Dataset generation

We're going to generate lots of example RGB-D images paired with grasps from a "ground truth" dataset.  We will be using the grasps in the dataset `data/grasps/robotiq_85_sampled_grasps.json` and a few candidate base worlds (table, box, and shelf). Let's start with `table.xml` for now.  First, we will generate world models by sampling object placements at random, then generating simulated RGB-D images, and finally generate grasp attribute images which we will later attempt to predict in Problem 2.

### Problem 1.A

In `gen_grasp_worlds`, you will need to implement a world sampler.  Use the stable faces logic of MP2, plus collision checking to generate worlds in which the objects rest in reasonable poses and are not colliding. The code provided for you will then save the world to `generated_worlds/world_XXXX.xml`. 

To visualize the worlds you've generated, run `klampt_browser generated_worlds/world_XXXX.xml`, or just run `klampt_browser` and click on each world file.  Note that you will need to run the program from the MP7 directory for the paths to be loaded properly.

![Example output](example_output/problem1a.png)


### Problem 1.B

Next, we will generate camera images from a simulated camera.  The basic implementation will just take one image per scene.  We would like our predictor to be invariant to color variations, so your task is to generate images with varying color.  You might also want to make the predictor less sensitive to noise in depth.  Modify the image generator to create at least 10 variations of the same scene, taken from the same viewpoint.

After uncommenting `PROBLEM='1b'` you can see your result.  Also, you can enter the PROBLEM string on the command line, e.g., `python problem1.py 1b`.  The result should be a set of files looking like this:

![Example output](example_output/color_0000_var0000.png)
![Example output](example_output/depth_0000_var0000.png)

In the written answers below, describe what variations you included in the dataset, and include the generated color and depth PNG files for ONE of your worlds and TWO of your variations.

In later stages of this project, you may also decide to add more viewpoint variations to make your predictor less sensitive to the particular viewpoint being used here.  The camera poses are predefined for you in the `resources` directory; if you want to edit these poses or set new poses you can uncomment out the "continue" marked under the TODO.



### Problem 1.C

Now, we will use the grasp database as "ground truth" labels for generating grasp attribute maps (images that give the per-pixel grasp attribute).  There are four attributes that we wish to predict:
- Grasp score (as predefined in the Grasp datastructure)
- Opening width
- Jaw opening axis, heading in camera frame
- Jaw opening axis, elevation in camera frame 

Note that since the axis is symmetric, we can choose either orientation.  Use the convention that the heading goes from 0 to 180 degrees, and the elevation goes from -90 to 90 degrees corresponding to a heading with y positive.

In `make_grasp_map`, implement a method that takes the sampled grasps and construct a grasp attribute map.  Rather than projecting a single grasp point to a pixel, you should spread out the attributes of sampled grasps to neighboring pixels so that the learning predicts similar scores at nearby points.  Also, make sure your attributes are limited to the range [0,1].

![Example output](example_output/grasp_0000_score.png)
![Example output](example_output/grasp_0000_opening.png)
![Example output](example_output/grasp_0000_axis_theta.png)
![Example output](example_output/grasp_0000_axis_elevation.png)

In the written answers below, include the generated PNG files for ONE of your worlds.  Describe possible issues that your labeling might pose for machine learning.

## Problem 2: Feature Generation and Machine Learning

In `problem2.py`, you will perform learning with the generated dataset using [Scikit-learn](https://scikit-learn.org/stable/).  This is a popular machine learning package which is quite easy to use.  (It is possible also to use PyTorch or TensorFlow for deep learning, but these often involve complex installation steps.)

The student who produces the most sensible predictions will be given extra credit!

### Problem 2.A

In `make_patch_dataset`, you will extract some number of example points from the image dataset to use for learning. Each example point will consist of a feature vector $x_i$ and a target value $y_i$, and we will predict $y=f(x)$.  You will need to settle upon a *feature representation*; that is, a method to extract a vector representation $x(color,depth,camera_xform,pixel)$ of the image characteristics affecting the label $y$ of the given pixel. 

The current implementation just selects 100 pixels at random and then returns a patch of the color and depth images around each sampled pixel.  There are two issues with this method:

- This has a high likelihood of oversampling low-quality regions and undersampling high quality grasps.
- Using raw depth and color might bias the predictor to use 

You should balance the sampling to include more pixels with higher quality grasps, and play around with what features you decide to include in your dataset.  You might also tune the patch size.  

Now that we've constructed a dataset, it's time to run some learning algorithms.  Implement some learning technique in `train_predictor`.  By default, we include a simple pipeline that first performs Principal Components Analysis (PCA) to reduce the dimension of your image patch to a much smaller vector, then performs a linear regression.  A few other possible pipelines are commented out.

Along with your feature selection explorations in Problem 2.A, choose a learning technique that produces relatively low testing error in comparison to predicting the average value.  Understanding these models and their hyperparameters may require investigating the documentation of Scikit-learn.  Also, you can use a [model selection technique](https://scikit-learn.org/stable/modules/grid_search.html) to help you select good model hyperparameters.

In your written answers below, describe how  you balanced sampling, what features you included in your dataset, and what ultimate model you used.  How well does that model compare against predicting the average, and against plain PCA + linear regression?


### Problem 2.B

Now, you will try using your model and feature extraction model to predict the score over a whole image.  Implement `predict_patches` to predict the score for all the pixels in `pts` using your existing model. Keep in mind that you will need to extract exactly the same features!

Note that `python problem2.py 2b` will run very, very slowly since it's running on every pixel.  Wait for one image to be output to the `predictions` folder and include it in your written answers below.

![Example output](example_output/image_0000.png)

### Problem 2.C 

Here you will produce a more efficient predictor that only predicts at the pixel level for a few pixels.  To minimize the number of pixels that are actually considered at the pixel level, we will build a hierarchy of *image patch score predictors*, which each take a patch as input and predict the score of the best grasp point within the patch.  These patch predictors will quickly eliminate many areas of an image with poor scores.

Train such a predictor that works on 80x80 patches, and another that works on 20x20 patches. This requires implementing `gen_partitioned_image_features` to divide an image into such patches.

### Problem 2.D

In `predict_scores_faster`, use the patch-based predictors to score an entire image, only selecting the top X% of the 80x80 patches to break into 20x20 patches, and then select the top Y% of the 20x20 patches to break into pixel-level predictions.  Tune X% and Y% to predict most of the high quality grasps at the pixel level, while avoiding pixel-level prediction as much as possible.  (It should take less than a minute to process a whole image)

In your images, output the score of 80x80 patches as 25% of the predicted max score within the patch, and output the score of 20x20 patches as 50% of the predicted max score within the patch.

![Example output](example_output/image_0000_fast.png)

Include one such image from the `predictions` folder in your written answers below.

## Problem 3: Predicting other attributes and reconstructing grasps

### Problem 3.A

Test your entire learning pipeline again, but with worlds and images generated from all three use cases (modify `problem1.py` to do so).  If you are confident in your pipeline, you may wish to generate more worlds and variations at this point.

In your written answers below, describe how the (pixel-level) prediction accuracy changes when you include more variation in the worlds.

Also, in `problem2.py`, try training your (pixel-level) models on the other grasp attributes (opening, axis_heading, axis_elevation).  Report the training and testing RMSEs compared to a constant prediction below.

### Problem 3.B (IR2 section only)

In `problem3.py`, use your grasp predictor to implement an image-based grasp planner. In the `generate` method, extract out some high scoring pixels and generate an `AntipodalGrasp` object accordingly.  Note that you will need to use all four grasp attributes to generate this object.

Test your grasp planner using the 'p' key.  Press 's' to sample a new arrangement of objects for further testing.

Take a snapshot of your grasp planner output and show it in the written answers below.

![Example output](example_output/problem3b.png)

(It's normal for the predictions to be quite poor; we're not using a lot of training data and our models are not terribly powerful.)

## Written responses

### Written response for Problem 1.B

Put your answer here.  Don't forget to include your images.

### Written response for Problem 1.C

Put your answer here.  Don't forget to include your images.

### Written response for Problem 2.A

Put your answer here.

### Written response for Problem 2.B

Put your predicted image here.

### Written response for Problem 2.D

Put your predicted image here.

### Written response for Problem 3.A

Put your answer here.

### Written response for Problem 3.B

Put your image here.